In [1]:
import findspark
findspark.init('C:\Spark\spark-3.0.1-bin-hadoop2.7')
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('nlp').getOrCreate()

In [2]:
data = spark.read.csv("SMSSpamCollection.csv",
                      inferSchema=True,
                      sep='\t')

In [3]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



In [4]:
from pyspark.sql.functions import length

data = data.withColumn('length', length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [5]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [6]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, 
                                CountVectorizer, IDF, 
                                StringIndexer)

tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [7]:
from pyspark.ml.feature import VectorAssembler
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], 
                           outputCol='features')

In [8]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

In [9]:
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(
    stages=[
        ham_spam_to_numeric,
        tokenizer,
        stop_remove,
        count_vec,
        idf,
        clean_up
    ]
)

In [10]:
cleaner = data_prep_pipe.fit(data)

In [11]:
clean_data = cleaner.transform(data)

In [12]:
clean_data = clean_data.select('label', 'features')

In [13]:
clean_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows



In [14]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [15]:
spam_detector = nb.fit(training)

In [16]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [17]:
test_results = spam_detector.transform(test)

In [18]:
test_results.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,3,9,1...|[-570.43563887307...|[0.99999999999988...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-998.60148177921...|[1.0,5.2287053578...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-821.93326970384...|[1.0,2.6513693580...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1184.0443352569...|[1.0,1.1709156686...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-672.31164568583...|[1.0,2.5287466941...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()

In [20]:
acc = acc_eval.evaluate(test_results)

In [21]:
print('ACC of NB model')
print(acc)

ACC of NB model
0.9236527829271867
